This ipynb is hyperparameter and modification version of:

1. https://www.kaggle.com/code/hiro5299834/store-sales-ridge-voting-bagging-et-bagging-rf
2. https://www.kaggle.com/code/andrej0marinchenko/hyperparamaters
3. https://www.kaggle.com/code/javi23ruiz/eda-with-plotly-useful-conclusions

Thanks for upvoting them as well.

In [2]:
#misc imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

# Purpose of the competition

* The entreprise is to set forcasts in 15 days horizon for each of the 54 stores of the Ecuadorian-based grocery retailer "Corporación Favorita".
* The predicted sales will be valued by the 'Root Mean Squared Logarithmic Error'.
* A full description of training data is provided [here](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data)
 

# Data engineering

## Getting Data

In [3]:
#Data list
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [4]:
#Train
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv',
                    parse_dates = ['date'], infer_datetime_format = True,
                    dtype = {'store_nbr' : 'category',
                             'family' : 'category'},
                    usecols = ['date', 'store_nbr', 'family', 'sales'])
train['date'] = train.date.dt.to_period('D')
train = train.set_index(['date', 'store_nbr', 'family']).sort_index()
train.head()

sales
date       store_nbr family           
2013-01-01 1         AUTOMOTIVE    0.0
                     BABY CARE     0.0
                     BEAUTY        0.0
                     BEVERAGES     0.0
                     BOOKS         0.0

In [5]:
#test
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv',
                   parse_dates = ['date'], 
                   infer_datetime_format = True)
test['date'] = test.date.dt.to_period('D')
test = test.set_index(['date', 'store_nbr', 'family']).sort_values('id')
test.head()

id  onpromotion
date       store_nbr family                          
2017-08-16 1         AUTOMOTIVE  3000888            0
                     BABY CARE   3000889            0
                     BEAUTY      3000890            2
                     BEVERAGES   3000891           20
                     BOOKS       3000892            0

## Dependant DATA (y)

In [6]:
#training horizon
sdate = '2017-04-30' # Start and end of training date
edate = '2017-08-15'

y = train.unstack(['store_nbr', 'family']).loc[sdate:edate]
y.head()

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-04-30        3.0       0.0    0.0     995.0   1.0      139.507   
2017-05-01        0.0       0.0    2.0     825.0   0.0      116.339   
2017-05-02        2.0       0.0    2.0    3179.0   0.0      447.238   
2017-05-03        5.0       0.0    6.0    2479.0   1.0      434.029   
2017-05-04        3.0       0.0    1.0    2454.0   0.0      438.214   

                                                 ...                     \
store_nbr                                        ...         9            
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES    MEATS   
date                                             ...                      
2017-04-30         2.0    208.0  315.0   60.114  ...       5.0  415.572   
2017-05-01         2.0    227.0  326.0   52.673  ...       5.0  603.395   
2017-05-02        20.0   1061.0  897.0  172.269  ...       0.0  495.275   
2017-05-03        22.0   1117.0  927.0  165.995  ...       4.0  386.662   
2017-05-04        15.0    956.0  755.0  133.780  ...       1.0  601.754   

                                                                        \
store_nbr                                                                
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS  POULTRY   
date                                                                     
2017-04-30         678.0         10.0                    16.0  513.866   
2017-05-01         950.0         19.0                    18.0  615.898   
2017-05-02         744.0         10.0                    13.0  346.344   
2017-05-03         513.0          5.0                    11.0  432.579   
2017-05-04         487.0          4.0                    13.0  312.411   

                                                                        
store_nbr                                                               
family     PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                    
2017-04-30        118.588  1762.442                        1.0  21.522  
2017-05-01        175.991  2502.085                        1.0  54.162  
2017-05-02        105.046  2739.654                        5.0  21.362  
2017-05-03         88.384  1442.451                        4.0  16.125  
2017-05-04        104.670  1285.772                        4.0  11.476  

[5 rows x 1782 columns]

## Independant DATA (X)

In [7]:
#Calendar
calendar = pd.DataFrame(index = pd.date_range('2013-01-01', '2017-08-31')).to_period('D')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv',
                  parse_dates = ['date'], infer_datetime_format = True,
                  index_col = 'date').to_period('D')
oil['avg_oil'] = oil['dcoilwtico'].rolling(7).mean()
calendar = calendar.join(oil.avg_oil)
calendar['avg_oil'].fillna(method = 'ffill', inplace = True)
calendar.dropna(inplace = True)

In [8]:
# Adding lags to oil
n_lags = 14 # Generaly people refill every week or two
for l in range(1, n_lags + 1) :
    calendar[f'oil_lags{l}'] = calendar.avg_oil.shift(l)
calendar.dropna(inplace = True)
calendar.head()

,avg_oil,oil_lags1,oil_lags2,oil_lags3,oil_lags4,oil_lags5,oil_lags6,oil_lags7,oil_lags8,oil_lags9,oil_lags10,oil_lags11,oil_lags12,oil_lags13,oil_lags14
2013-01-24,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,93.470000,93.284286,93.284286,93.284286,93.218571
2013-01-25,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,93.470000,93.284286,93.284286,93.284286
2013-01-26,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,93.470000,93.284286,93.284286
2013-01-27,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,93.470000,93.284286
2013-01-28,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,93.470000


In [9]:
#Holidays
hol = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv',
                  parse_dates = ['date'], 
                  infer_datetime_format = True,
                  index_col = 'date').to_period('D')
hol = hol[hol.locale == 'National'] # I'm only taking National holiday so there's no false positive.
hol = hol.groupby(hol.index).first() # Removing duplicated holiday at the same date
hol.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


In [10]:
calendar = calendar.join(hol) # Joining calendar with holiday dataset
calendar['dofw'] = calendar.index.dayofweek # Weekly day
calendar['wd'] = 1
calendar.loc[calendar.dofw > 4, 'wd'] = 0 # If it's saturday or sunday then it's not Weekday
calendar.loc[calendar.type == 'Work Day', 'wd'] = 1 # If it's Work Day event then it's a workday
calendar.loc[calendar.type == 'Transfer', 'wd'] = 0 # If it's Transfer event then it's not a work day
calendar.loc[calendar.type == 'Bridge', 'wd'] = 0 # If it's Bridge event then it's not a work day
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = 0 # If it's holiday and the holiday is not transferred then it's holiday
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True), 'wd'] = 1 # If it's holiday and transferred then it's not holiday
calendar = pd.get_dummies(calendar, columns = ['dofw'], drop_first = True) # One-hot encoding (Make sure to drop one of the columns by 'drop_first = True')
calendar = pd.get_dummies(calendar, columns = ['type']) # One-hot encoding for type holiday (No need to drop one of the columns because there's a "No holiday" already)
calendar.drop(['locale', 'locale_name', 'description', 'transferred'], axis = 1, inplace = True) # Unused columns
calendar.head()

,avg_oil,oil_lags1,oil_lags2,oil_lags3,oil_lags4,oil_lags5,oil_lags6,oil_lags7,oil_lags8,oil_lags9,...,dofw_3,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
2013-01-24,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,...,1,0,0,0,0,0,0,0,0,0
2013-01-25,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,...,0,1,0,0,0,0,0,0,0,0
2013-01-26,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,...,0,0,1,0,0,0,0,0,0,0
2013-01-27,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,...,0,0,0,1,0,0,0,0,0,0
2013-01-28,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,...,0,0,0,0,0,0,0,0,0,0


In [11]:
school_season = [] # Feature for school fluctuations
for i, r in calendar.iterrows() :
    if i.month in [4, 5, 8, 9] :
        school_season.append(1)
    else :
        school_season.append(0)
calendar['school_season'] = school_season
calendar.head()

,avg_oil,oil_lags1,oil_lags2,oil_lags3,oil_lags4,oil_lags5,oil_lags6,oil_lags7,oil_lags8,oil_lags9,...,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
2013-01-24,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,93.490000,...,0,0,0,0,0,0,0,0,0,0
2013-01-25,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,93.644286,...,1,0,0,0,0,0,0,0,0,0
2013-01-26,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,93.970000,...,0,1,0,0,0,0,0,0,0,0
2013-01-27,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,...,0,0,1,0,0,0,0,0,0,0
2013-01-28,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,94.331429,...,0,0,0,0,0,0,0,0,0,0


In [12]:
fourier = CalendarFourier(freq = 'W', order = 4)
dp = DeterministicProcess(index = y.index,
                          order = 1,
                          seasonal = False,
                          constant = False,
                          additional_terms = [fourier],
                          drop = True)
x = dp.in_sample()
x = x.join(calendar)
x.head()

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)",avg_oil,oil_lags1,oil_lags2,...,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
date,,,,,,,,,,,,,,,,,,,,,
2017-04-30,1.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,49.358571,49.358571,49.358571,...,0,0,1,0,0,0,0,0,0,1
2017-05-01,2.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,49.154286,49.358571,49.358571,...,0,0,0,0,0,0,1,0,0,1
2017-05-02,3.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969,48.870000,49.154286,49.358571,...,0,0,0,0,0,0,0,0,0,1
2017-05-03,4.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,48.711429,48.870000,49.154286,...,0,0,0,0,0,0,0,0,0,1
2017-05-04,5.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,48.187143,48.711429,48.870000,...,0,0,0,0,0,0,0,0,0,1


In [13]:
xtest = dp.out_of_sample(steps = 16) # 16 because we are predicting next 16 days
xtest = xtest.join(calendar)
xtest.head()

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)",avg_oil,oil_lags1,oil_lags2,...,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
2017-08-16,109.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,48.281429,48.648571,48.934286,...,0,0,0,0,0,0,0,0,0,1
2017-08-17,110.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,47.995714,48.281429,48.648571,...,0,0,0,0,0,0,0,0,0,1
2017-08-18,111.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521,47.852857,47.995714,48.281429,...,1,0,0,0,0,0,0,0,0,1
2017-08-19,112.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490,47.852857,47.852857,47.995714,...,0,1,0,0,0,0,0,0,0,1
2017-08-20,113.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,47.852857,47.852857,47.852857,...,0,0,1,0,0,0,0,0,0,1


# Model

## Creation

In [25]:
from joblib import Parallel, delayed
import warnings

# Import necessary library
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# SEED for reproducible result
SEED = 5

class CustomRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):
    
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES': # Because SCHOOL AND OFFICE SUPPLIES has weird trend, we use decision tree instead.
            
            model = XGBRegressor(booster = "dart",
                                 tree_method = 'exact',
                                 objective = 'reg:tweedie',
                                 sample_type = 'weighted',
                                 normalize_type = 'forest',
                                 eval_metric = 'tweedie-nloglik@1.5',
                                 tweedie_variance_power = 1.5,
                                 random_state = SEED)

                                         
        else:
            param_grid = {'alpha': np.arange(0.1, 10, 0.1),
                         'fit_intercept': [True, False]}
            model = GridSearchCV(Ridge(),
                                 param_grid,
                                 refit= True)
           
        
        model.fit(X, y)
        return model

    def fit(self, X, y):
        from tqdm.auto import tqdm
        
        
        if self.verbose == 0 :
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        else :
            print('Fit Progress')
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in tqdm(range(y.shape[1])))
        return
    
    def predict(self, X):
        from tqdm.auto import tqdm
        if self.verbose == 0 :
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in self.estimators_)
        else :
            print('Predict Progress')
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in tqdm(self.estimators_))
        
        return np.stack(y_pred, axis=1)

## training

In [29]:
%%time

model = CustomRegressor(n_jobs=-1, verbose=1)
model.fit(x, y) #it takes a while -59min 42s-

y_pred = pd.DataFrame(model.predict(x), index=x.index, columns=y.columns)

Predict Progress


  0%|          | 0/1782 [00:00<?, ?it/s]

## Evaluation

In [30]:
from sklearn.metrics import mean_squared_log_error

y_pred = y_pred.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

#RMSLE per product familly
y_['pred'] = y_pred.values
print('Total RMSLE : ', y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))

#Total RMSLE
print('Total RMSLE : ', np.sqrt(np.sqrt(mean_squared_log_error(y_['sales'], y_['pred']))))

Total RMSLE :  family
AUTOMOTIVE                    0.687956
BABY CARE                     0.499984
BEAUTY                        0.684595
BEVERAGES                     0.404491
BOOKS                         0.351372
BREAD/BAKERY                  0.367345
CELEBRATION                   0.713779
CLEANING                      0.521186
DAIRY                         0.351035
DELI                          0.393118
EGGS                          0.519130
FROZEN FOODS                  0.497113
GROCERY I                     0.384531
GROCERY II                    0.737812
HARDWARE                      0.700430
HOME AND KITCHEN I            0.670757
HOME AND KITCHEN II           0.646263
HOME APPLIANCES               0.601448
HOME CARE                     0.437647
LADIESWEAR                    0.667713
LAWN AND GARDEN               0.636117
LINGERIE                      0.765667
LIQUOR,WINE,BEER              0.735881
MAGAZINES                     0.677933
MEATS                         0.405962
PER

## Predict

In [31]:
ypred = pd.DataFrame(model.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)
ypred = ypred.stack(['store_nbr', 'family'])
ypred.head()

Predict Progress


  0%|          | 0/1782 [00:00<?, ?it/s]

sales
           store_nbr family                 
2017-08-16 1         AUTOMOTIVE     4.183704
                     BABY CARE      0.000000
                     BEAUTY         6.029726
                     BEVERAGES   2238.785128
                     BOOKS          0.177138

# Submission

In [32]:
sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred.values
sub.to_csv('submission.csv', index = False) # Submit
sub

,id,sales
0,3000888,4.183704
1,3000889,0.000000
2,3000890,6.029726
3,3000891,2238.785128
4,3000892,0.177138
...,...,...
28507,3029395,298.389901
28508,3029396,98.863386
28509,3029397,1075.590919
28510,3029398,152.634125
